# Una sola Funcion que hace todo(todo el poder al alcance de un un click)

## LOAD 3 EXCELS BOOKS

In [1]:
import pandas as pd
import numpy as np
import Actu_Colums as Ac
import win32com.client as win32
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.worksheet.table import Table, TableStyleInfo
import datetime


## Cargo el Capex

In [4]:
PathofE = 'archvis/CAPEX/Pendiente de entrada al 27.11.2023.xlsx' 
Hoja= 'DATA' 
CPX = pd.read_excel(PathofE, sheet_name= Hoja)

## Cargo el PAP Completo


In [20]:
papxls= 'archvis/PAP/PAP 28.11.xlsx'
PAP = pd.read_excel(papxls, sheet_name='Hoja2') 

## Cargo el PEA_old

In [21]:
PEA_old_Path = 'archvis/PEA_O/PEA 27.11.2023.xlsx'

In [22]:
PEA_old = pd.read_excel(PEA_old_Path, sheet_name= 'DATA') 

# Funcion Ultimate

In [23]:
def updateEA(PEA_NW,PAP,PEA_old):
    
    #Pre-procesing of CPX
    CPX = Ac.convert_columns_to_str(PEA_NW.copy(), ['DOC_PREC', 'POS_PREC']) #Convert to str a key column
    CPX["OC Posición"] = CPX["DOC_PREC"].str.cat(CPX["POS_PREC"], sep= ":") # format
    CPX_EI = CPX.loc[CPX['SOLICITANTE'] == 'Eduardo Iberico']#Filter
    Monto_In = CPX_EI["EN_PROC_USD"].sum() #Mont of USD 
    
    #Filter of PEA_old
    PEA_oldE = PEA_old.loc[PEA_old['SOLICITANTE'] == 'Eduardo Iberico'] #less work 
    PEA_oldE_F = PEA_oldE[['CONCATENADO', 'PAP', 'SITE', 'ESTADO PAP', 'ESTADO EA', 'COMENTARIO', 'RESPONSABLE EA']]
    #Here I merge for inherit the information from the old dataframe
    EA_act = Ac.merge_dataframes(CPX_EI, PEA_oldE_F,'CONCATENADO', index=CPX_EI.index)  
    #Preprocessing of PAP
    PAP_f = PAP[['OC Posición','N° Sol','SIte','Estado','# Días']]
    PAP_f = PAP_f.rename(columns={'N° Sol': 'PAP', #Rename
       'SIte': 'SITE',
       'Estado': 'ESTADO PAP'})
    
    
    #I update the columns that are inherited from PEA_old with PAP
    EA_act['# Días'] = np.nan
    EA_act =  Ac.update_values(EA_act, PAP_f, "OC Posición",['PAP','SITE','ESTADO PAP','# Días']) #Use my own function
    Monto_out = EA_act["EN_PROC_USD"].sum() #Verify mont in USD
    if (Monto_In == Monto_out): #Conditional hows say me if the amount in USD isn´t modify
        print("all ok")
    else: print ("something is wrong")

    ##Create a DF of posibles EA (recente data act)
    df_filtrado = EA_act[(EA_act['SOLICITANTE'] == 'Eduardo Iberico') & 
                        (EA_act['ESTADO EA'] == 'Pendiente')] 
                                                            
    P_EA = df_filtrado.sort_values('# Días') 

    
    ## All the process be enbebed in the big dataframe with out filters ###

    #Do a copy of Capex 
    PEA_CL = PEA_NW.copy()
    PEA_CL = PEA_NW.rename(columns={'NUM_SOT': 'PAP', #rename
                                    'NATURALEZA': 'SITE',
                                    'BLOQUEO': 'ESTADO PAP',
                                    'FECHA_VENTA' : 'ESTADO EA' })
    ############################################################################################
    PEA_CL['COMENTARIO'] = np.nan
    PEA_CL['RESPONSABLE EA'] = np.nan
    PEA_CL.update(EA_act) #Update in big df

    PEA_CL = Ac.crear_area(PEA_CL) #asigno area por analista 
    PEA_CL['CONCATENADO'] = PEA_CL['CONCATENADO'].astype(str) 
    PEA_CL['DOC_PREC'] = PEA_CL['DOC_PREC'].astype(str) #4 better format in excel
    #########################################################################################

    #Format data.tipe
    cols = ['FECHA_DOC','fecha reporte']
    PEA_CL = Ac.date_format(PEA_CL,cols)
    PEA_CL['FECHA_DOC'] = pd.to_datetime(PEA_CL['FECHA_DOC'],dayfirst=True).dt.date
    
    #I label all new rows as pending and normalise 
    PEA_CL['ESTADO EA'] = PEA_CL['ESTADO EA'].apply(lambda x: 'Pendiente' if pd.isnull(x) 
                                                or x == '' else ('Ejecutada' if x == 'EJECUTADA' else x))

    print("Proceso Concluido, Listo para exportar al excel");
        
    
    return PEA_CL,P_EA #return the final DF and the most recently data for analytics

In [85]:
EA,df = updateEA(CPX,PAP,PEA_old)

todo bien
Proceso Concluido, Listo para exportar al excel


### Format 4 excel and the table in it

In [8]:
Today_str = datetime.date.today().strftime('%d-%m-%y')
Today_str
Today_path = 'Resultados/EAUPDATE{}.xlsx'.format(Today_str[0:5])
Today_path
# Exporto a excel
EA.to_excel(Today,index=False)
### format for excel result  ###########################
columns_usseles = ['T', 'AM', 'AO',  #I hide columns that are not useful
                   'AI', 'AJ', 'X', 'N', 
                   'Y', 'Z', 'AA','AN', 'AB']

# Load Excel
wb = load_workbook(Today_path)
ws = wb.active

# Create a table in Excel with the DataFrame data
tab = Table(displayName="Tabla1", ref="$A$1:$AY${}".format(len(EA)+1))
for col in columns_usseles:
    ws.column_dimensions[col].hidden = True

# Style of table
style = TableStyleInfo(name="TableStyleMedium9", showFirstColumn=False,
                       showLastColumn=False, showRowStripes=True, showColumnStripes=False)
tab.tableStyleInfo = style
ws.add_table(tab)
wb.save(Today_path)


'Resultados/EAUPDATE01-12.xlsx'

### Analysis by criteria


In [86]:
df_notnull = df[df['# Días'].notnull()]

### By Analista 

In [ ]:
df_notnull[df_notnull['RESPONSABLE EA'] == 'JHORDAN'].head(50)

### By PAP estatus

In [27]:
PosiblesEa = ['FAC', 'PAC', 'Aprobaciones PAC']
df_notnull[df_notnull['ESTADO PAP'].isin(PosiblesEa)]

,CONCATENADO,DOC_PREC,POS_PREC,FECHA_DOC,Estrategia de Liberación,Cod.Solicitante,SOLICITANTE,TIPO_DOC,TIPO DE MATERIAL,MATERIAL,...,BLOQUEO,FECHA_VENTA,OC Posición,PAP,SITE,ESTADO PAP,ESTADO EA,COMENTARIO,RESPONSABLE EA,# Días
4461,4500663437110,4500663437,110,2023-10-18,Pendiente de Liberación,PREDESGT027U,Eduardo Iberico,Pedidos,SERVICIO,3006030,...,NaN,NaN,4500663437:110,56298,REQUE_ETEN,FAC,Pendiente,NaN,LAURA,0.0
2550,450065608910,4500656089,10,2023-09-19,Pendiente de Liberación,PREDESGT027U,Eduardo Iberico,Pedidos,SERVICIO,3006030,...,NaN,NaN,4500656089:10,55218,PAITA,FAC,Pendiente,NaN,JHORDAN,0.0
4468,450066343740,4500663437,40,2023-10-18,Pendiente de Liberación,PREDESGT027U,Eduardo Iberico,Pedidos,SERVICIO,3006030,...,NaN,NaN,4500663437:40,56302,LEGUIA,FAC,Pendiente,NaN,LAURA,0.0
4466,450066343760,4500663437,60,2023-10-18,Pendiente de Liberación,PREDESGT027U,Eduardo Iberico,Pedidos,SERVICIO,3006030,...,NaN,NaN,4500663437:60,56303,ITURRI,FAC,Pendiente,NaN,LAURA,0.0
4465,450066343770,4500663437,70,2023-10-18,Pendiente de Liberación,PREDESGT027U,Eduardo Iberico,Pedidos,SERVICIO,3006030,...,NaN,NaN,4500663437:70,56304,CARAZ,FAC,Pendiente,NaN,LAURA,0.0
364,4500627223180,4500627223,180,2023-05-25,Pendiente de Liberación,PREDESGT027U,Eduardo Iberico,Pedidos,SERVICIO,3009529,...,NaN,NaN,4500627223:180,49386,SAPILLICA,FAC,Pendiente,8/11: EJECUTADO POR LIQUIDAR,ANGGIE,3.0
2060,450065287210,4500652872,10,2023-09-08,Pendiente de Liberación,PREDESGT027U,Eduardo Iberico,Pedidos,SERVICIO,3009526,...,NaN,NaN,4500652872:10,55406,CERRO_PUMA,PAC,Pendiente,NaN,ANGGIE,4.0
473,4500626519310,4500626519,310,2023-05-23,Pendiente de Liberación,PREDESGT027U,Eduardo Iberico,Pedidos,SERVICIO,3000732,...,NaN,NaN,4500626519:310,54299,FEDERICO_VILLAREAL_CHICLAYO,FAC,Pendiente,8/11: EJECUTADO POR LIQUIDAR,ANGGIE,5.0
2151,450065170840,4500651708,40,2023-09-04,Pendiente de Liberación,PREDESGT027U,Eduardo Iberico,Pedidos,SERVICIO,3000731,...,NaN,NaN,4500651708:40,54411,TERMINAL_BAGUA,FAC,Pendiente,NaN,JHORDAN,5.0
1680,450065000510,4500650005,10,2023-08-23,Pendiente de Liberación,PREDESGT027U,Eduardo Iberico,Pedidos,SERVICIO,3009526,...,NaN,NaN,4500650005:10,54537,SHAGUINDO,FAC,Pendiente,NaN,ANGGIE,5.0


### By info

In [28]:
df_comentado = df.dropna(subset=['COMENTARIO'])
df_comentado_N0 = df_comentado[df_comentado['COMENTARIO'] != '0']


In [72]:
df_comentado_N0.tail(50)

,CONCATENADO,DOC_PREC,POS_PREC,FECHA_DOC,Estrategia de Liberación,Cod.Solicitante,SOLICITANTE,TIPO_DOC,TIPO DE MATERIAL,MATERIAL,...,BLOQUEO,FECHA_VENTA,OC Posición,PAP,SITE,ESTADO PAP,ESTADO EA,COMENTARIO,RESPONSABLE EA,# Días
276,450062233050,4500622330,50,2023-05-11,Pendiente de Liberación,PREDESGT027U,Eduardo Iberico,Pedidos,SERVICIO,3000732,...,NaN,NaN,4500622330:50,51211,LAS_LOMAS,Visita ejecutada,Pendiente,8/11: EJECUTADO POR LIQUIDAR,ANGGIE,60.0
280,450062233010,4500622330,10,2023-05-11,Pendiente de Liberación,PREDESGT027U,Eduardo Iberico,Pedidos,SERVICIO,3000732,...,NaN,NaN,4500622330:10,51539,GUAYAQUIL,Visita ejecutada,Pendiente,8/11: EJECUTADO POR LIQUIDAR,ANGGIE,60.0
725,450063117410,4500631174,10,2023-06-15,Pendiente de Liberación,PREDESGT027U,Eduardo Iberico,Pedidos,SERVICIO,3000752,...,NaN,NaN,4500631174:10,53502,TAYABAMBA,Pre registro,Pendiente,2023-11-30 00:00:00,DANNER,63.0
857,450063468910,4500634689,10,2023-06-23,Pendiente de Liberación,PREDESGT027U,Eduardo Iberico,Pedidos,SERVICIO,3000752,...,NaN,NaN,4500634689:10,53495,TAYABAMBA,Pre registro,Pendiente,EN PROCESO,DANNER,63.0
265,4500622330160,4500622330,160,2023-05-11,Pendiente de Liberación,PREDESGT027U,Eduardo Iberico,Pedidos,SERVICIO,3000732,...,NaN,NaN,4500622330:160,51045,PANAMERICANA_QUIRUVILCA,FAC,Pendiente,8/11: EJECUTADO POR LIQUIDAR,ANGGIE,66.0
869,450063511130,4500635111,30,2023-06-27,Pendiente de Liberación,PREDESGT027U,Eduardo Iberico,Pedidos,SERVICIO,3000752,...,NaN,NaN,4500635111:30,53375,NAT_TALARA_P6,Pre registro,Pendiente,0,DEMETRIO,67.0
862,450063510630,4500635106,30,2023-06-27,Pendiente de Liberación,PREDESGT027U,Eduardo Iberico,Pedidos,SERVICIO,3000752,...,NaN,NaN,4500635106:30,53377,NAT_VIRU_P2,Pre registro,Pendiente,0,DEMETRIO,67.0
279,450062233020,4500622330,20,2023-05-11,Pendiente de Liberación,PREDESGT027U,Eduardo Iberico,Pedidos,SERVICIO,3000732,...,NaN,NaN,4500622330:20,51206,HUAMACHUCO_CENTRO,PAC,Pendiente,8/11: EJECUTADO POR LIQUIDAR,ANGGIE,68.0
475,4500626519290,4500626519,290,2023-05-23,Pendiente de Liberación,PREDESGT027U,Eduardo Iberico,Pedidos,SERVICIO,3000732,...,NaN,NaN,4500626519:290,52055,CONCEJO_LEONARDO_ORTIZ,PAC,Pendiente,0,ANGGIE,70.0
379,450062750640,4500627506,40,2023-05-26,Pendiente de Liberación,PREDESGT027U,Eduardo Iberico,Pedidos,SERVICIO,3000731,...,NaN,NaN,4500627506:40,53211,JAYANCA_CENTRO,Pre registro,Pendiente,Pendiente de revisión de Jhordan la liquidación,JHORDAN,74.0


### By Money

In [88]:
df_montos = df[['CONCATENADO','EN_PROC_USD','PAP', 'SITE', 'COMENTARIO','RESPONSABLE EA', '# Días']].sort_values('EN_PROC_USD',ascending=0)
df_montos.head(50)

,CONCATENADO,EN_PROC_USD,PAP,SITE,COMENTARIO,RESPONSABLE EA,# Días
857,450063468910,139505.237975,53495,TAYABAMBA,EN PROCESO,DANNER,63.0
1790,450065083510,67911.412658,-,TP6276-TALARA,X anular /20/11,DANNER,NaN
4456,450066331810,66784.716456,-,TA3878_CHIPTA,NaN,DEMETRIO,NaN
1845,450065119810,58353.174684,-,CAC0111 - NUEVO CAC CHICLAYO,NaN,Marlon,NaN
5106,450066725670,55634.612658,-,TA3878-CHIPTA,2023-12-31 00:00:00,JORGE,NaN
725,450063117410,42110.977215,53502,TAYABAMBA,2023-11-30 00:00:00,DANNER,63.0
2958,450065949250,39835.189873,-,TA3881-TARUSCANCHA,NaN,JORGE,NaN
722,450063098210,39180.146835,-,TC5857-CAJABAMBA,Liquidado-Pendiente de cargar EA,JORGE,NaN
4752,450066535030,37720.253165,-,TJ2484-QUIRUVILCA,NaN,DEMETRIO,NaN
2479,450065503430,36678.511392,56034,NODO CHICLAYO-PROVINCIA,NaN,DANNER,6.0


## Automatizar agregar el DF a la Plantilla de EA 

### Usando win32

In [71]:
excel = win32.gencache.EnsureDispatch('Excel.Application')

In [72]:
wb = excel.Workbooks.Open('D:/EA/BASE/PlantillaV2.xlsx')

In [73]:
ws = wb.Worksheets.Add()
ws.Name = 'DATA1'
excel.Visible = True


In [74]:
datos = EA.values.tolist()
df_copy = EA.copy()

df_copy['FECHA_DOC'] = pd.to_datetime(df_copy['FECHA_DOC'])
df_copy['fecha reporte'] = pd.to_datetime(df_copy['fecha reporte'])
df_copy['COMENTARIO'] = df_copy['COMENTARIO'].astype(str)

# Convierte los objetos datetime.datetime en objetos pywintypes.Time
df_copy['FECHA_DOC'] = df_copy['FECHA_DOC'].apply(lambda x: pywintypes.Time(x.to_pydatetime()))
df_copy['fecha reporte'] = df_copy['fecha reporte'].apply(lambda x: pywintypes.Time(x.to_pydatetime()))
# Escribe los datos en la hoja de trabajo
ws.Range(ws.Cells(1, 1), ws.Cells(len(datos), len(datos[0]))).Value = datos

C:\Users\C26764\AppData\Local\Temp\ipykernel_6272\1454075142.py:5: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_copy['fecha reporte'] = pd.to_datetime(df_copy['fecha reporte'])


TypeError: must be a pywintypes time object (got datetime.date)

In [67]:
for column in df_copy.columns:
    if df_copy[column].apply(lambda x: isinstance(x, datetime.date)).any():
        print(column)

FECHA_DOC
fecha reporte


### Usando metodo tradicional openpyxl

In [76]:
archivo_excel = 'archvis/Plantilla/PlantillaV2.xlsx'

# Escribir el DataFrame a una nueva hoja de Excel
with pd.ExcelWriter(archivo_excel, engine='openpyxl', mode='a') as writer:
    EA.to_excel(writer, sheet_name='DATA1',index=False,if_sheet_exists='replace')


C:\Users\C26764\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Slicer List extension is not supported and will be removed
  warn(msg)
C:\Users\C26764\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
